In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import Image, display
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
tf.__version__

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

In [ ]:
!mkdir models

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 24
num_epochs = 100
val_steps = 20
MODEL_NAME = 'densenet169'

In [ ]:
image_csv = pd.read_csv('./../data/all.csv')

# Prepend image filenames in train/ with relative path
filenames = image_csv['path'].tolist()
labels = image_csv['label'].tolist()

train_filenames, val_filenames, train_labels, val_labels = train_test_split(filenames,
                                                                            labels,
                                                                            train_size=0.9,
                                                                            random_state=420)

num_train = len(train_filenames)
num_val = len(val_filenames)
output_size = len(set(train_labels + val_labels))

In [ ]:
def _parse_fn(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    image_normalized = (tf.cast(image_decoded, tf.float32)/127.5) - 1
    image_resized = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
    return image_resized, label

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((tf.constant(train_filenames), tf.constant(train_labels))).map(_parse_fn).shuffle(buffer_size=10000).batch(BATCH_SIZE)


val_data = tf.data.Dataset.from_tensor_slices((tf.constant(val_filenames), tf.constant(val_labels))).map(_parse_fn).batch(BATCH_SIZE)


In [ ]:
# Base model with Xception
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

model = tf.keras.applications.densenet.DenseNet169(input_shape=IMG_SHAPE, 
                                       classes=output_size,
                                       weights=None)

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('./models/weights_%s_%s.h5' % (MODEL_NAME, IMAGE_SIZE), monitor='val_acc', 
                     mode='max', verbose=0, save_best_only=True)
csv_logger = CSVLogger('logs/%s.log' % MODEL_NAME)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss='categorical_crossentropy',
             metrics=['acc'])

model.summary()


In [ ]:
steps_per_epoch = round(num_train)//BATCH_SIZE

In [ ]:
history = model.fit(train_data.repeat(),
                    epochs=num_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data=val_data.repeat(), 
                    validation_steps=val_steps,
                    callbacks=[es, mc, csv_logger])

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
